## Use Case: Log Request

In [1]:
import pickle
import struct
from abc import ABC, abstractmethod

# Base Command class using pickle for serialization.
class Command(ABC):
    @abstractmethod
    def execute(self):
        pass

    def store(self):
        # Serialize the command object using pickle.
        return pickle.dumps(self)
    
    @staticmethod
    def load(data):
        # Deserialize the command object using pickle.
        return pickle.loads(data)

# Inventory class representing our stock system.
class Inventory:
    def __init__(self):
        self.stock = {}  # Dictionary mapping product names to quantities.
    
    def add_stock(self, product, quantity):
        self.stock[product] = self.stock.get(product, 0) + quantity
        print(f"Added {quantity} of {product}. New stock: {self.stock[product]}")
    
    def remove_stock(self, product, quantity):
        if product not in self.stock:
            print(f"Product {product} does not exist in inventory.")
            return
        if self.stock[product] < quantity:
            print(f"Not enough {product} in stock to remove {quantity}. Current stock: {self.stock[product]}")
            return
        self.stock[product] -= quantity
        print(f"Removed {quantity} of {product}. New stock: {self.stock[product]}")

    def __str__(self):
        return str(self.stock)

# Concrete Command: Add stock operation.
class AddStockCommand(Command):
    def __init__(self, product, quantity, inventory=None):
        self.product = product
        self.quantity = quantity
        self.inventory = inventory  # Not serialized; assigned at runtime.
    
    def execute(self):
        if self.inventory is not None:
            self.inventory.add_stock(self.product, self.quantity)

# Concrete Command: Remove stock operation.
class RemoveStockCommand(Command):
    def __init__(self, product, quantity, inventory=None):
        self.product = product
        self.quantity = quantity
        self.inventory = inventory  # Not serialized; assigned at runtime.
    
    def execute(self):
        if self.inventory is not None:
            self.inventory.remove_stock(self.product, self.quantity)

# CommandLog uses the Command's store/load methods and writes each command
# with a 4-byte length prefix so that the stored binary data is properly separable.
class CommandLog:
    def __init__(self, filename):
        self.filename = filename

    def store_command(self, command):
        data = command.store()
        # Write a 4-byte unsigned integer representing the length, then the data.
        with open(self.filename, 'ab') as file:
            file.write(struct.pack('I', len(data)))
            file.write(data)
        print(f"Stored command: {command}")
    
    def load_commands(self):
        commands = []
        try:
            with open(self.filename, 'rb') as file:
                while True:
                    # Read the 4-byte length prefix.
                    len_bytes = file.read(4)
                    if not len_bytes:
                        break  # End of file.
                    length = struct.unpack('I', len_bytes)[0]
                    data = file.read(length)
                    command = Command.load(data)
                    commands.append(command)
                    print(f"Loaded command: {command}")
        except FileNotFoundError:
            print("Log file not found.")
        return commands

def main():
    # Create an inventory instance.
    inventory = Inventory()
    command_log = CommandLog('inventory_commands.pkl')
    
    # Create and execute an add-stock command for "Widget".
    add_widget = AddStockCommand("Widget", 100, inventory)
    add_widget.execute()
    command_log.store_command(add_widget)
    
    # Create and execute an add-stock command for "Gadget".
    add_gadget = AddStockCommand("Gadget", 50, inventory)
    add_gadget.execute()
    command_log.store_command(add_gadget)
    
    # Create and execute a remove-stock command for "Widget".
    remove_widget = RemoveStockCommand("Widget", 20, inventory)
    remove_widget.execute()
    command_log.store_command(remove_widget)
    
    print("\nInventory before recovery:", inventory)
    
    # Simulate a system crash by reinitializing the inventory.
    inventory = Inventory()
    print("Inventory after crash (cleared):", inventory)
    
    # Recover commands from the log and re-execute them.
    commands = command_log.load_commands()
    for command in commands:
        # Reassign the current inventory instance to the command.
        command.inventory = inventory
        command.execute()
    
    print("Inventory after recovery:", inventory)

if __name__ == "__main__":
    main()

Added 100 of Widget. New stock: 100
Stored command: <__main__.AddStockCommand object at 0x00000279C1861850>
Added 50 of Gadget. New stock: 50
Stored command: <__main__.AddStockCommand object at 0x00000279C1861580>
Removed 20 of Widget. New stock: 80
Stored command: <__main__.RemoveStockCommand object at 0x00000279C06D6DC0>

Inventory before recovery: {'Widget': 80, 'Gadget': 50}
Inventory after crash (cleared): {}
Loaded command: <__main__.AddStockCommand object at 0x00000279C1840AF0>
Loaded command: <__main__.AddStockCommand object at 0x00000279C1840AC0>
Loaded command: <__main__.RemoveStockCommand object at 0x00000279C1840F10>
Added 100 of Widget. New stock: 100
Added 50 of Gadget. New stock: 50
Removed 20 of Widget. New stock: 80
Inventory after recovery: {'Widget': 80, 'Gadget': 50}
